In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .getOrCreate()

22/07/05 10:33:55 WARN Utils: Your hostname, msi-ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.22.178 instead (on interface wlo1)
22/07/05 10:33:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/05 10:33:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.option("header",True).option("inferSchema", True).csv('/home/visal/Desktop/address_parsing/address_parsing_NER_model/corpus/data/archive/training_address_v2_no_kh.csv')

In [19]:
df = df.withColumn('new_house_no', F.regexp_extract('Address', \
    '^#.*?(?=,)', 0))

In [23]:
df  = df.withColumn('new_street_no', F.regexp_extract('Address', \
    'St\..*?(?=,)', 0))

In [34]:
df = df.withColumn('new_province', F.regexp_extract('Address', \
    '[^,]+$', 0)).withColumn('new_province', F.regexp_replace(F.col('new_province'), '^\s', ''))

In [37]:
df.columns

['Address',
 'house_no',
 'street_no',
 'village_phum',
 'commune_khum',
 'district_srok',
 'city_province',
 'new_house_no',
 'new_street_no',
 'new_province']

In [38]:
new_df = df.select('Address', 'new_house_no', 'new_street_no', 'village_phum', 'commune_khum', 'district_srok', 'new_province')

In [40]:
new_df1 = new_df.select([F.when(F.col(c)=="",None).otherwise(F.col(c)).alias(c) for c in new_df.columns])

In [45]:
new_df1.toPandas().to_csv("address_v3.csv", index=False)